In [1]:
import pandas as pd
import json

## 各个DataFrame数据导出函数
1. json_user_info中解决可能存在的数据重复问题，同一用户不同城市的问题
2. 考虑可以将这个模块整体组成的函数的一个参数设置成需要的特征，但是这需要大幅更改内部函数的输出

## 改进型读取程序：

In [42]:
dict1 = {'a':{'a':1,'b':2,'c':3}}
df1 = pd.DataFrame.from_dict(dict1, orient = 'index')

In [43]:
display(df1)

,a,b,c
a,1,2,3


In [28]:
user_features = ['Account', 'Account Created', 'Default Baby Birthday', 'Baby Count']
user_features_notuserprop = ['city', 'region', 'country', 'device_manufacturer', 'device_type', 'apk_channel',
                             'start_apk_channel', 'version_name']
event_purchase_features = ['UseCoupon', 'CouponPrice', 'CouponType']
event_coupon_features = ['Gain Coupon', 'Use Coupon Click', 'Unused Coupon View']

In [51]:
def update_data(user_map, json_object, feature_list_toplevel, feature_list_eventproperty, feature_list_userproperty):
    for feature in feature_list_toplevel:
        if feature in json_object:
            user_map[feature] = json_object[feature]
    if json_object['event_type'] == 'revenue_amount':
        for feature in feature_list_eventproperty:
            if feature in json_object['event_properties']:
                user_map[feature] = json_object['event_properties'][feature]
    for feature in feature_list_userproperty:
        if feature in json_object['user_properties']:
            user_map[feature] = json_object['user_properties'][feature]
    if json_object['event_type'] == 'Gain Coupon':
        user_map['Gain Coupon'] = True
    elif json_object['event_type'] == 'Usecoupon Click':
        user_map['Use Coupon Click'] = json_object['event_properties']['Source']
    elif json_object['event_type'] == 'Unused Coupon View':
        user_map['Unused Coupon View'] = True

In [52]:
data_map = {}

In [53]:
import os
json_path = '../data/raw/raw_json/'
json_folder = os.listdir(json_path)
for json_file_path in json_folder:
    json_file = open(json_path + json_file_path)
    if json_file_path[-4:] == 'json':
        for json_biobject in json_file:
            json_object = json.loads(json_biobject)
            if json_object['user_id'] in data_map:
                update_data(data_map[json_object['user_id']], json_object, user_features_notuserprop, 
                           event_purchase_features, user_features)
            else:
                data_map[json_object['user_id']] = {}
                update_data(data_map[json_object['user_id']], json_object, user_features_notuserprop, 
                           event_purchase_features, user_features)


In [54]:
df_user_features = pd.DataFrame.from_dict(data_map, orient = 'index')

In [55]:
display(df_user_features[~df_user_features['UseCoupon'].isnull()])
display(df_user_features.shape)

,city,region,country,device_manufacturer,device_type,version_name,Account,Account Created,Default Baby Birthday,Baby Count,UseCoupon,CouponPrice,CouponType,Gain Coupon,Unused Coupon View,Use Coupon Click
3790cf7892b049608c881d3145fc2c7f,Chengdu,Sichuan,China,Apple,Apple iPad Air 2,7.14.2,wechat,2018-09-13 00:52:00 +0000,201506,1.0,False,NA,NA,NaN,NaN,NaN
0d9e02ee71c2429fa29d8f3916741df9,Nanjing,Jiangsu,China,Apple,Apple iPad Air 2,7.13.2,wechat,2019-01-19 09:15:44 +0000,201404,1.0,False,NA,NA,NaN,True,couponlist
82a09287ed1846d896d3f3e4a5ac4fbd,Nanning,Guangxi,China,Apple,Apple iPhone 6s,7.14.1,mobile,2016-07-31 15:35:29 +0000,201403,1.0,True,150,Bundle,True,True,NaN
95f37fc38a14485f97c5800a0a5e6ba7,Linyi,Shandong,China,Apple,Apple iPad 4,7.14.2,wechat,2019-02-28 13:01:56 +0000,201404,1.0,True,150,Bundle,True,True,NaN
9d92770bf5d74066aca5787e79a010e7,Chengdu,Sichuan,China,Apple,Apple iPad 5,7.14.2,wechat,2018-03-18 05:40:20 +0000,201105,1.0,False,NA,NA,NaN,NaN,NaN
6933194953fe43ad99092332825242b5,Hefei,Anhui,China,Apple,Apple iPhone 7 Plus,7.14.2,wechat,2018-01-03 03:50:30 +0000,201711,4.0,True,150,Bundle,True,True,couponlist
b07a3f6da7d1484ba3076e6cdb9b3615,Ningbo,Zhejiang,China,Apple,Apple iPhone 8 Plus,7.14.2,mobile,2019-02-26 23:53:56 +0000,201412,1.0,False,NA,NA,NaN,True,couponlist
ec76b8596baf4bfbbf242cbd98f23ab8,Guangzhou,Guangdong,China,Apple,Apple iPhone 8 Plus,7.14.2,wechat,2018-12-01 12:27:29 +0000,201606,1.0,True,150,Bundle,True,True,couponlist
c5c0ce8e5a9f43edbda0fd0b8e1430d7,Shanghai,Shanghai,China,Apple,Apple iPhone X,7.14.2,mobile,2018-07-26 15:35:34 +0000,201608,1.0,False,NA,NA,NaN,NaN,NaN
feaa538d22f0495ebb91cf9f2f2e545c,Nanjing,Jiangsu,China,Apple,Apple iPhone 6s Plus,7.14.0,wechat,2018-11-17 14:23:54 +0000,201111,1.0,False,NA,NA,NaN,NaN,NaN


(191359, 16)

In [14]:
print(data_map['5ea91969b0d34eb99987cc8b4f36f0e7'])

{'city': 'Anhe', 'region': 'Shanxi', 'country': 'China', 'device_manufacturer': 'Apple', 'device_type': 'Apple iPhone XS Max', 'version_name': '7.14.1', 'Account': 'mobile', 'Account Created': '2017-11-06 13:59:42 +0000', 'Default Baby Birthday': '201607', 'Baby Count': 1}


In [56]:
df_user_features.to_csv('user_data_v2.csv')